# Написатор хокку

In [ ]:
#импорты

from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
#кладем в перемнную текст
TRAIN_TEXT_FILE_PATH = 'Басё.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text = text_file.readlines()
text = ' '.join(text)

text=text[235:]
text[:600]

'Где же ты, кукушка?\n  Вспомни, сливы начали цвести,\n  Лишь весна дохнула\n   В хижине, отстроенной после пожара \n  Слушаю, как градины стучат.\n  Лишь один я здесь не изменился,\n  Словно этот старый дуб.\n  Ива склонилась и спит,\n  И кажется мне, соловей на ветке -\n  Это её душа\n  Только дохнёт ветерок -\n  С ветки на ветку ивы\n  Бабочка перепорхнёт.\n  Как завидна их судьба!\n  К северу от суетного мира\n  Вишни зацвели в горах.\n  Разве вы тоже из тех,\n  Кто не спит, опьянён цветами,\n  О мыши на чердаке?\n  Дождь в тутовой роще шумит…\n  На земле едва шевелится\n  Больной шелковичный червь.\n  Ещё на ос'

Токенизация: присвоение числового индекса каждому символу в тексте. В результате получим два словаря: для кодировки букв, знаков препинания и проч в числа - и обратно.

In [ ]:
def txt_to_seq(text):
    char_counts = sorted(Counter(text).items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]

    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    print(char_to_idx)

    sequence = np.array([char_to_idx[char] for char in text])

    return sequence, char_to_idx, idx_to_char

In [ ]:
sequence, char_to_idx, idx_to_char = txt_to_seq(text)

{' ': 0, 'о': 1, 'е': 2, 'а': 3, 'н': 4, 'и': 5, '\n': 6, 'т': 7, 'с': 8, 'л': 9, 'р': 10, 'к': 11, 'в': 12, 'д': 13, 'у': 14, '.': 15, 'м': 16, 'п': 17, 'ь': 18, 'й': 19, 'я': 20, 'ы': 21, 'г': 22, 'з': 23, 'б': 24, 'ч': 25, ',': 26, 'х': 27, 'ж': 28, 'ш': 29, 'В': 30, '!': 31, 'П': 32, 'ц': 33, 'С': 34, 'ю': 35, '-': 36, 'О': 37, 'К': 38, 'Н': 39, 'ё': 40, 'щ': 41, 'Т': 42, 'Л': 43, 'И': 44, 'Б': 45, '…': 46, 'З': 47, 'Д': 48, 'М': 49, 'э': 50, 'У': 51, '?': 52, '—': 53, 'Я': 54, 'Х': 55, 'Г': 56, 'Р': 57, '«': 58, '»': 59, 'Ж': 60, 'А': 61, 'Е': 62, ':': 63, 'Ш': 64, 'Э': 65, 'Ц': 66, 'Ф': 67, 'Ч': 68, '“': 69, '”': 70, 'ф': 71}


In [ ]:
sequence[:50]

array([56, 13,  2,  0, 28,  2,  0,  7, 21, 26,  0, 11, 14, 11, 14, 29, 11,
        3, 52,  6,  0,  0, 30,  8, 17,  1, 16,  4,  5, 26,  0,  8,  9,  5,
       12, 21,  0,  4,  3, 25,  3,  9,  5,  0, 33, 12,  2,  8,  7,  5])

Формируем батчи - "куски" последовательности, на которых модель будет обучаться. Величина батча = 32 знака. Модель будет предсказывать +1 смвол относительно текста, который получает на вход.

In [ ]:
BATCH_LEN = 128
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for i in range(BATCH_SIZE):
        start = np.random.randint(0, len(sequence) - BATCH_LEN)
        chunk = sequence[start: start + BATCH_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

4. Генерация текста. Каждую итерацию модель предскажет по 1 букве.

In [ ]:
def evaluate(RNN, char_to_idx, idx_to_char, start_text=text[0], prediction_len=200, temp=0.35):
    hidden = RNN.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = RNN(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = RNN(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
        if i>=0.8*prediction_len and predicted_char=='\n':
          break

    return predicted_text

5. Класс RNN. Принимает на вход эмбединг, возвращает вероятность для каждого последующего символа

In [ ]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

6. Обучаем нейросеть.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = RNN(input_size=len(idx_to_char), hidden_size=256, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 6000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        print()
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)
        print('* '*10)

Loss: 3.285726308822632

Ге
   аок еее ле оре кене кее летк во ле но тда ано олоо ао ае оне ве са ое оае поде села ане сто водлри
   поа  отло  оа ресве то вна се тое аио поно ое ои оет дора оинит  аоио оово ло опо  оте вод во
* * * * * * * * * * 
Loss: 2.6013354301452636

Голебот вето токо чозтой ветно вогной понодий вотоно полосто ветот вата
  О пемет вет ветет водом детна
  В ветоней сородитой вотона
  О раденне волет водет стом сороль.

* * * * * * * * * * 
Loss: 2.3294831848144533

Г-
  Сролетний винь
  Постол полет полит
  В родене но слусне коротоно.
  В ветет и пороком
  Соладь полет венет веть.
  Но сетны пролона.
  Пузаниний саленний калет порока.

* * * * * * * * * * 
Loss: 2.1828905534744263

Гит вето в одтер
  Подо солнадо в молет остер.
  В под до продонь код ветол полет.
  В серонно стересть пруда.
  В продний сраватит остол.
   Под серонний ветер,

* * * * * * * * * * 
Loss: 2.042214689254761

Гет пода одомой,
  Вод плет сверот водомой,
  В сере додного сторок!
  О бале

In [ ]:
torch.save(model, 'Haiku.pth')

In [ ]:
model_pred = torch.load('Haiku.pth')

7. Генерация текста

In [ ]:
model_pred.eval()
print(evaluate(
    model_pred,
    char_to_idx,
    idx_to_char,
    temp=0.35,
    prediction_len=400,
    start_text='Лю'
    )
)

Люби в душе моей,
  О вишни старого сада!
  С шелестом облетели
  Горных роз лепестки...
  Дальний шум водопада.
  Посадили деревья в саду.
  Тихо, тихо, чтоб их ободрить,
  Шепчет осенний дождь.
  Может быть, кости мои
  Выбелит ветер... Он в сердце
  Холодом мне дохнул.
  По горной тропинке иду.
  Вдруг стало мне отчего-то легко.

